In [2]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [3]:
from pycaret.datasets import get_data
dataset = get_data('insurance')

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
from pycaret.regression import *
exp_reg101 = setup(data = dataset, target = 'charges', session_id=123)

,Description,Value
0,Session id,123
1,Target,charges
2,Target type,Regression
3,Original data shape,"(1338, 7)"
4,Transformed data shape,"(1338, 10)"
5,Transformed train set shape,"(936, 10)"
6,Transformed test set shape,"(402, 10)"
7,Numeric features,3
8,Categorical features,3
9,Preprocess,True


In [6]:
model=compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2701.9927,23548981.3626,4832.9682,0.8320,0.4447,0.3137,0.2150
rf,Random Forest Regressor,2771.4583,25416502.3827,5028.6343,0.8172,0.4690,0.3303,0.4310
lightgbm,Light Gradient Boosting Machine,2992.1828,25521038.3331,5042.0978,0.8149,0.5378,0.3751,0.3820
et,Extra Trees Regressor,2833.3624,28427844.2412,5305.6516,0.7991,0.4877,0.3363,0.3280
ada,AdaBoost Regressor,4316.0568,29220505.6498,5398.4561,0.7903,0.6368,0.7394,0.1060
xgboost,Extreme Gradient Boosting,3278.3216,31628234.0000,5599.0988,0.7709,0.5733,0.4208,0.2440
br,Bayesian Ridge,4311.2349,38391950.0874,6176.8896,0.7306,0.5910,0.4447,0.0870
llar,Lasso Least Angle Regression,4303.7694,38386824.2786,6176.4846,0.7306,0.5952,0.4434,0.0860
lr,Linear Regression,4303.5559,38388058.4578,6176.5920,0.7306,0.5949,0.4433,0.8260
lar,Least Angle Regression,4303.5559,38388058.4578,6176.5920,0.7306,0.5949,0.4433,0.0860


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [7]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.drop('charges', axis=1), dataset['charges'], test_size=0.2, random_state=42)

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1070 entries, 560 to 1126
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1070 non-null   int64  
 1   sex       1070 non-null   object 
 2   bmi       1070 non-null   float64
 3   children  1070 non-null   int64  
 4   smoker    1070 non-null   object 
 5   region    1070 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 58.5+ KB


In [13]:
X_train.region.value_counts()

,count
region,
southeast,283
northeast,267
southwest,264
northwest,256


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [16]:
X_train.smoker.unique()

array(['no', 'yes'], dtype=object)

In [15]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1070 entries, 560 to 1126
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1070 non-null   int64  
 1   sex       1070 non-null   object 
 2   bmi       1070 non-null   float64
 3   children  1070 non-null   int64  
 4   smoker    1070 non-null   object 
 5   region    1070 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 58.5+ KB


In [19]:
tr01=ColumnTransformer(transformers=[("scaler",StandardScaler(),[0,2,3]),("enco",OrdinalEncoder(categories=[['no', 'yes']]),[4])
,("ohe",OneHotEncoder(drop="first"),[1,5])],remainder="passthrough")

In [20]:
from sklearn.feature_selection import SelectKBest,f_regression

In [21]:
tr02=SelectKBest(f_regression,k=5)

In [22]:

from sklearn.ensemble import GradientBoostingRegressor


In [23]:
tr03=GradientBoostingRegressor()

In [24]:
pipe=Pipeline(steps=[("tr01",tr01),("tr02",tr02),("tr03",tr03)])

In [25]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tr01',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  [0, 2, 3]),
                                                 ('enco',
                                                  OrdinalEncoder(categories=[['no',
                                                                              'yes']]),
                                                  [4]),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first'),
                                                  [1, 5])])),
                ('tr02',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x786f76131a20>)),
                ('tr03', GradientBoostingRegressor())])

In [27]:
y_predict=pipe.predict(X_test)

In [29]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [30]:
r2_score(y_test,y_predict)

0.8783243162579033

In [31]:
from sklearn.model_selection import GridSearchCV

In [38]:
parameters={"tr02__k":[3,6]}

In [39]:
gird=GridSearchCV(estimator=pipe,param_grid=parameters,scoring='r2',cv=5)

In [40]:
gird.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tr01',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scaler',
                                                                         StandardScaler(),
                                                                         [0, 2,
                                                                          3]),
                                                                        ('enco',
                                                                         OrdinalEncoder(categories=[['no',
                                                                                                     'yes']]),
                                                                         [4]),
                                                                        ('ohe',
                                                                         OneHotEncoder(drop='first'),
                                                                         [1,
                                                                          5])])),
                                       ('tr02',
                                        SelectKBest(k=5,
                                                    score_func=<function f_regression at 0x786f76131a20>)),
                                       ('tr03', GradientBoostingRegressor())]),
             param_grid={'tr02__k': [3, 6]}, scoring='r2')

In [41]:
gird.best_params_

{'tr02__k': 6}

In [42]:
gird.best_score_

0.8392519788181497